In [ ]:
# Loading libraries:
suppressMessages({    
    library(data.table)
    library(dplyr)
    library(ggplot2)
    library(IRdisplay)
    library(phyloseq)
    library(plyr)
    library(stringr)
    library(tidyverse)
    library(vegan)
})

# Reading KOs table and transforming into a phyloseq object

In [ ]:
# Reading table
data <- read.table("presabs_kos.tsv", header = TRUE, sep = "\t", row.names = 1)
otumat <- as.matrix(data[-1])
OTU = otu_table(otumat, taxa_are_rows = TRUE)
#otumat

In [ ]:
sampledata <- data.frame(sample_types = sub("^[^_]*_", "", colnames(otumat)))
rownames(sampledata) = colnames(otumat)
head(sampledata)

SAM = sample_data(sampledata)

In [ ]:
physeq = phyloseq(OTU, SAM)

In [ ]:
physeq

# Statistical analysis of groups comparison

## Betadisper and Adonis2

In [ ]:
metadata = data.frame(sample_data(physeq))
css_beta = distance(physeq, method="bray")
ado_result = adonis2(css_beta ~ sample_types, data = metadata, perm=1e3)
ado_result

In [ ]:
bd = betadisper(css_beta, metadata$'sample_types')
anova(bd)

In [ ]:
pdf("disp_kos_presabs.pdf", width = 18, height = 5)
boxplot(bd)
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
plot(bd)

In [ ]:
permutest(bd)

## Pairwise ANOSIM

In [ ]:
cbn <- combn(x = unique(metadata$sample_types), m = 2)
p <- c()  # vector to store p values
R <- c()  # vector to store R values

for (i in 1:ncol(cbn)) {
    ps.subs <- subset_samples(physeq, sample_types %in% cbn[, i])
    metadata_sub <- data.frame(sample_data(ps.subs))
    permanova_pairwise <- anosim(phyloseq::distance(ps.subs, method="bray"), 
                                 metadata_sub$sample_types)
    p <- c(p, permanova_pairwise$signif[1])  # Store p-values
    R <- c(R, permanova_pairwise$statistic)  # Store R statistics
}

p.adj <- p.adjust(p, method = "BH")  # Adjust p-values
p.table <- cbind.data.frame(t(cbn), R = R, p = p, p.adj = p.adj)  # Combine all results in a table


In [ ]:
write.table(p.table, "pairwise_anosim.tsv", row.names = FALSE, col.names = TRUE, quote = FALSE, sep = "\t")

## Pairwise ANOSIM heatmap

### Criteria for R-value color codes:
R-value Near 0:
R ~ 0.0 to 0.2: This range typically suggests that there is little to no observable difference between the groups. The samples can be considered very similar in composition.

Moderate R-values:
R ~ 0.2 to 0.5: This range suggests moderate differentiation. Samples might be considered somewhat different, but the distinction isn't very strong. Interpretation in this range can depend on the sensitivity required in the study and the natural variability of the dataset.
High R-values:

R-value close to 1:
R > 0.5: This range indicates strong differentiation between samples. The higher the R-value, especially approaching or exceeding 0.7, the more distinct the community compositions between the groups. Samples with these R-values can be considered to have different compositions.


In [ ]:
library(pheatmap)
data <- read.table("pairwise_anosim.tsv", header = TRUE, sep = "\t")
# Building a matrix
samples <- unique(c(data$X1, data$X2))
r_matrix <- matrix(NA, nrow = length(samples), ncol = length(samples), 
                   dimnames = list(samples, samples))
for (i in 1:nrow(data)) {
    row <- which(samples == data$X1[i])
    col <- which(samples == data$X2[i])
    r_matrix[row, col] <- data$R[i]
    r_matrix[col, row] <- data$R[i]  # Ensure the matrix is symmetric
}

desired_order <- c("pic", "mfp", "deep", "1_sm", "5_sm", "10_sm", "15_sm", "20_sm", "1_bwa", "5_bwa", "10_bwa", "15_bwa", "20_bwa", "1_sho", "5_sho", "10_sho", "15_sho", "20_sho")
r_matrix <- r_matrix[desired_order, desired_order]

# Set the lower triangle to NA
r_matrix[lower.tri(r_matrix)] <- NA

# Define the colors and breaks
colors <- c(
  colorRampPalette(c("#a80202", "#ff6403"))(20),    # Gradient from 0 to 0.2
  colorRampPalette(c("#ff6403", "#ffda05"))(30),    # Sharp change from 0.2 to 0.5
  colorRampPalette(c("#ffda05", "black"))(50)       # Sharp change from 0.5 to 1.0
)
breaks <- c(
  seq(0, 0.2, length.out = 21),     # Breaks from 0 to 0.2
  seq(0.2, 0.5, length.out = 31)[-1],   # Breaks from 0.2 to 0.5 (excluding duplicate 0.2)
  seq(0.5, 1.0, length.out = 51)[-1]    # Breaks from 0.5 to 1.0 (excluding duplicate 0.5)
)

# Plot the heatmap
pdf("rvalues_hp.pdf", width = 7, height = 5)
pheatmap(r_matrix,
         color = colors,
         breaks = breaks,
         cluster_rows = FALSE,  # Disable clustering to preserve the order
         cluster_cols = FALSE,  # Disable clustering to preserve the order
         show_rownames = TRUE,
         show_colnames = TRUE,
         display_numbers = FALSE,
         na_col = "white")  # Set the color for NA values
dev.off()


# Ordination plots

In [ ]:
# Adding new groups to phyloseq object metadata
sample_names <- colnames(otumat)

# Initialize an empty dataframe for metadata
new_sampledata <- data.frame(sample_name = sample_names, sample_group = character(length(sample_names)), stringsAsFactors = FALSE)

# Function to determine sample_type based on the suffix
get_sample_type <- function(sample_name) {
  suffix <- sub("^[^_]*_", "", sample_name)
  if (suffix %in% c("1_sm", "5_sm")) {
    return("low_sm")
  } else if (suffix %in% c("10_sm", "15_sm", "20_sm")) {
    return("high_sm")
  } else if (suffix %in% c("1_bwa", "5_bwa")) {
    return("low_bwa")
  } else if (suffix %in% c("10_bwa", "15_bwa", "20_bwa")) {
    return("high_bwa")
  } else if (suffix %in% c("1_sho", "5_sho")) {
    return("low_sho")
  } else if (suffix %in% c("10_sho", "15_sho", "20_sho")) {
    return("high_sho")
  } else if (suffix == "deep") {
    return("deep")
  } else if (suffix == "mfp") {
    return("mfp")
  } else if (suffix == "pic") {
    return("pic")
  } else {
    return(NA)
  }
}

# Apply the function to determine the sample_type for each sample_name
new_sampledata$sample_group <- sapply(new_sampledata$sample_name, get_sample_type)
rownames(new_sampledata) = colnames(otumat)

sample_data(physeq)$'sample_group' = new_sampledata$sample_group

head(sample_data(physeq))
metadata = data.frame(sample_data(physeq))

In [ ]:
colors_list <- c(
  "#000000",
  "#E6194B",
  "#3CB44B",
  "#0082C8",
  "#F58231",
  "#911EB4",
  "#46F0F0",
  "#F032E6",
  "#D2F53C",
  "#FABED4",
  "#008080",
  "#DCBEFF",
  "#808080"
)

colorCode_sample <- c(
  "1_bwa" = "#000000",
  "5_bwa" = "#E6194B",
  "10_bwa" = "#3CB44B",
  "15_bwa" = "#0082C8",
  "20_bwa" = "#F58231",
  "1_sm" = "#911EB4",
  "5_sm" = "#46F0F0",
  "10_sm" = "#F032E6",
  "15_sm" = "#D2F53C",
  "20_sm" = "#FABED4",
  "deep" = "#008080",
  "mfp" = "#DCBEFF",
  "pic" = "#808080"
)


colorCode_group <- c(
  "low_bwa" = "#D2F53C",
  "high_bwa" = "#F58231",
  "low_sm" = "#46F0F0",
  "high_sm" = "#F032E6",
  "deep" = "#000000",
  "mfp" = "#DCBEFF",
  "pic" = "#808080",
  "low_sho" = "#E6194B",
  "high_sho" = "#FFFF00"
)



In [ ]:
# Testing different ordination methods
dist = "bray"
ord_meths = c("DCA", "CCA", "RDA", "MDS", "PCoA")
plist = llply(as.list(ord_meths), function(i, physeq_obj, dist){
        ordi = ordinate(physeq_obj, method=i, distance=dist)
        plot_ordination(physeq_obj, ordi, "samples")
}, physeq, dist)

In [ ]:
names(plist) <- ord_meths
pdataframe = ldply(plist, function(x){
    df = x$data[, 1:2]
    colnames(df) = c("Axis_1", "Axis_2")
    return(cbind(df, x$data))
})
names(pdataframe)[1] = "method"

In [ ]:
# Displaying the plots with all the methods to choose the best
options(repr.plot.width=7, repr.plot.height=6)
print_plots <- function() {
    for (index in 1:5) {
        p = plist[[index]] + geom_point(size=2, alpha=1, aes(color=metadata$sample_group)) + 
            stat_ellipse(level=0.9, type="norm", geom="polygon", alpha=0, aes(color=metadata$sample_group)) +
            theme_bw() + 
            scale_color_manual(values=colorCode_group) +
            labs(color = "Sample groups") 
        print(p)
    }
}
print_plots()

In [ ]:
# Generating the final figure in pdf
options(repr.plot.width=6, repr.plot.height=5)
pdf("RDA_real_kos.pdf", width = 7, height = 6)
p = plist[[3]] + geom_point(size=2, alpha=1, aes(color=metadata$sample_group)) + 
    stat_ellipse(level=0.9, type="norm", geom="polygon", alpha=0, aes(color=metadata$sample_group)) +
    theme_bw() + 
    scale_color_manual(values=colorCode_group) +
    labs(color = "Sample groups") 
p
dev.off()

## Hierarchical clustering

In [ ]:
library(dendextend)
data <- read.table("presabs_kos.tsv", header = TRUE, sep = "\t", row.names = 1)
otumat <- as.matrix(data[-1])
sampledata <- data.frame(sample_types = sub("^[^_]*_", "", colnames(otumat)))
rownames(sampledata) = colnames(otumat)
colorCode <- c(
  "1_bwa" = "#000000",
  "5_bwa" = "#E6194B",
  "10_bwa" = "#3CB44B",
  "15_bwa" = "#0082C8",
  "20_bwa" = "#F58231",
  "1_sm" = "#911EB4",
  "5_sm" = "#46F0F0",
  "10_sm" = "#F032E6",
  "15_sm" = "#D2F53C",
  "20_sm" = "#FABED4",
  "deep" = "#008080",
  "mfp" = "#DCBEFF",
  "pic" = "#808080"
)
bc_dist <- vegan::vegdist(t(otumat), method = "bray")
ward <- as.dendrogram(hclust(bc_dist, method = "ward.D2"))
plot_data <- as.dendrogram(ward)
label_colors <- colorCode[sampledata$sample_types[order.dendrogram(plot_data)]]
labels_colors(ward) <- label_colors

pdf("real_kos_hclus.pdf", width = 20, height = 8)
par(mar = c(6, 2, 1, 1))
plot(ward)
legend("topright", inset = c(0, 0), legend = names(colorCode), fill = colorCode, title = "Sample Types", cex = 1.5)
dev.off()

write.table(labels(plot_data), "dendrogram_labels.txt", row.names = FALSE, col.names = FALSE, quote = FALSE)
